In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import statsmodels.api as sm

from typing import List

import sys
sys.path.append("../") 

from src.data.etl_layer import ETL
from src.data.dqc_layer import DQC
from src.data.eda_layer import EDA

from src.modelling.feature_extraction import Feature_Extraction as FE

In [2]:
# creating instances of the ETL class for each dataset and extracting data

item_categories_etl: ETL = ETL("../data/raw/item_categories.csv")
items_etl: ETL = ETL("../data/raw/items.csv")
sales_etl: ETL = ETL("../data/raw/sales_train.csv")
shops_etl: ETL = ETL("../data/raw/shops.csv")
test_etl: ETL = ETL("../data/raw/test.csv")


84 rows and 2 columns has been read from item_categories.csv

22170 rows and 3 columns has been read from items.csv

2935849 rows and 6 columns has been read from sales_train.csv

60 rows and 2 columns has been read from shops.csv

214200 rows and 3 columns has been read from test.csv


In [3]:
# receiving extracted data in the form of dataframes

item_categories_df: pd.DataFrame = item_categories_etl.get_data()
items_df: pd.DataFrame = items_etl.get_data()
sales_df: pd.DataFrame = sales_etl.get_data()
shops_df: pd.DataFrame = shops_etl.get_data()
test_df: pd.DataFrame = test_etl.get_data()

# DQC
* overview of the data (types of data, descriptive statistics and data examples) for each dataset and a more detailed analysis of the data of the 'sales_train.csv' dataset

In [ ]:
# creating instances of the DQC class for each DataFrame

item_categories_dqc: DQC = DQC(item_categories_df, item_categories_etl.df_title)
items_dqc: DQC = DQC(items_df, items_etl.df_title)
sales_dqc: DQC = DQC(sales_df, sales_etl.df_title)
shops_dqc: DQC = DQC(shops_df, shops_etl.df_title)
test_dqc: DQC = DQC(test_df, test_etl.df_title)

In [ ]:
item_categories_dqc.data_review()

In [ ]:
items_dqc.data_review()

In [ ]:
shops_dqc.data_review()

In [ ]:
test_dqc.data_review()

In [ ]:
sales_dqc.data_review()

In [ ]:
# outliers check (for item_price and item_cnt_day columns in the sales DataFrame)

columns_outliers: dict = sales_dqc.outliers_check(["item_price", "item_cnt_day"])

### Visualization of item_price and item_cnt_day columns of the sales DataFrame 

In [ ]:
# getting IQR boundaries for item_price values 
item_price_interval_border_1: np.float64 = columns_outliers["iqr_interval"]["item_price"][0]
item_price_interval_border_2: np.float64 = columns_outliers["iqr_interval"]["item_price"][1]

# getting item_price values excluding outliers
item_price_wo_outliers: pd.Series = sales_df.loc[(sales_df["item_price"] >= item_price_interval_border_1) & (sales_df["item_price"] <= item_price_interval_border_2)]["item_price"]

In [ ]:
# getting IQR boundaries for item_cnt_day values 
item_cnt_day_interval_border_1: np.float64 = columns_outliers["iqr_interval"]["item_cnt_day"][0]
item_cnt_day_interval_border_2: np.float64 = columns_outliers["iqr_interval"]["item_cnt_day"][1]

# getting item_cnt_day values excluding outliers
item_cnt_day_wo_outliers: pd.Series = sales_df.loc[(sales_df["item_cnt_day"] >= item_cnt_day_interval_border_1) & (sales_df["item_cnt_day"] <= item_cnt_day_interval_border_2)]["item_cnt_day"]

In [ ]:
# Visualization
fig, ax = plt.subplots(2, 2, figsize=(8, 8))
plt.subplots_adjust(hspace=0.4, wspace=0.3)

ax[0][0].hist(sales_df["item_price"], bins=32, color="lightcoral")
ax[0][1].hist(item_price_wo_outliers, bins=32, color="darkcyan")
ax[0][0].axvline(sales_df["item_price"].mean(), color="yellow")
ax[0][1].axvline(sales_df["item_price"].mean(), color="yellow")

ax[0][0].text(sales_df["item_price"].mean() - 12000, 1000, s=f"{sales_df['item_price'].mean() : .2f}", rotation=90, color="orange")
ax[0][1].text(sales_df["item_price"].mean() - 100, 200000, s=f"{sales_df['item_price'].mean() : .2f}", rotation=90, color="orange")

ax[0][0].legend(["mean", "frequency"])
ax[0][1].legend(["mean", "frequency"])

ax[0][0].set_title("item_price histogram with outliers")
ax[0][1].set_title("item_price histogram without outliers")

ax[0][0].set_xlabel("item price")
ax[0][1].set_xlabel("item price")
#
ax[1][0].hist(sales_df["item_cnt_day"], bins=32, color="lightcoral")
ax[1][1].hist(item_cnt_day_wo_outliers, bins=32, color="darkcyan")
ax[1][0].axvline(item_cnt_day_wo_outliers.mean(), color="yellow")
ax[1][1].axvline(item_cnt_day_wo_outliers.mean(), color="yellow")

ax[1][0].text(item_cnt_day_wo_outliers.mean() - 100, 1000, s=f"{sales_df['item_cnt_day'].mean() : .2f}", rotation=90, color="orange")
ax[1][1].text(item_cnt_day_wo_outliers.mean() - 0.05, 1500000, s=f"{item_cnt_day_wo_outliers.mean().mean() : .2f}", rotation=90, color="orange")

ax[1][0].legend(["mean", "frequency"])
ax[1][1].legend(["mean", "frequency"])

ax[1][0].set_title("item_cnt_day histogram with outliers")
ax[1][1].set_title("item_cnt_day histogram without outliers")

ax[1][0].set_xlabel("item amount per day")
ax[1][1].set_xlabel("item amount per day")

ax[0][0].set_yscale("symlog")
ax[1][0].set_yscale("symlog")

plt.show()

In [ ]:
# ids availability check
missing_shop_ids_idx: pd.Index = sales_df[~sales_df["shop_id"].isin(shops_df["shop_id"])].index
missing_item_ids_idx: pd.Index = sales_df[~sales_df["item_id"].isin(items_df["item_id"])].index
missing_item_categories_idx: pd.Index = items_df[~items_df["item_category_id"].isin(item_categories_df["item_category_id"])].index

print(f"Row indexes of missing sales_train item ids in items_df: {list(missing_item_ids_idx)}")
print(f"Row indexes of missing sales_train shop ids in shops_df: {list(missing_shop_ids_idx)}")
print(f"Row indexes of missing items_df item categories id in item_categories_df: {list(missing_item_categories_idx)}")

In [ ]:
# NA values check for sales_train DF 
_ = sales_dqc.na_values_check()

In [ ]:
# data consistency and data uniqueness check for sales_train DF 
for column in sales_df.columns:
    print(f"\nConsistency column: {column}")
    _ = sales_dqc.consistency_uniqueness_check([column])


In [ ]:
# data types check for sales_train DF 
_ = sales_dqc.types_check()

# ETL
* Processing the sales_train dataframe data and creating a new montnly_sales dataframe based on it

In [ ]:
# Processing the sales_train dataframe data 
sales_df_processed: pd.DataFrame = sales_etl.transform(["item_price", "item_cnt_day"], ["item_cnt_day"])

In [ ]:
# drop not available ids in the processed sales_train DF
sales_df_processed.drop(index=missing_item_ids_idx.union(missing_shop_ids_idx), inplace=True)

In [ ]:
# saving the processed sales dataframe in the csv file
sales_etl.load_data_csv("processed_sales_train")

In [ ]:
# saving the processed sales dataframe in the sqlite db
sales_etl.load_data_sqlite("processed_sales_train")

### Creating a monthly_sales dataframe based on the main sales_train
* It will include the monthly sales for each product for each store

In [ ]:
# adding an Year-month column 
sales_df_processed["month"]: pd.DataFrame = sales_df_processed["date"].dt.to_period("M")

In [ ]:
# grouping data to calculate monthly amount for a specific item in a specific store
monthly_sales: pd.DataFrame = FE(sales_df_processed).create_monthly_df()
# monthly_sales["month"] = monthly_sales["month"].dt.strftime("%Y-%m") # converting datetime type into str 
                                                                    # for data visualization (can be ignored)
# monthly_sales["month"] = monthly_sales["month"].astype("int64") # converting datetime type into str 


print(monthly_sales.shape)
monthly_sales.head()

In [ ]:
# creating instances of the DQC class for monthly_sales DF
monthly_sales_dqc: DQC = DQC(monthly_sales)

In [ ]:
monthly_sales_dqc.data_review()

In [ ]:
# creating instances of the DQC class for monthly_sales DF
monthly_sales_etl: ETL = ETL(df=monthly_sales)

In [ ]:
# Processing monthly_sales data
monthly_sales_processed: pd.DataFrame = monthly_sales_etl.transform(["item_cnt_month"], ["item_cnt_month"])

# EDA
* Exploratory data analysis for the original sales dataframe, for time series for all unique store-item pairs, and for time series for cumulative monthly and daily sales

In [ ]:
# creating instances of the EDA class for sales DF
sales_eda: EDA = EDA(sales_df_processed)

In [ ]:
# checking for correlation of sales dataframe features using the listed methods
sales_eda.features_corr_check(columns=["date_block_num", "shop_id", "item_id", "item_price", "item_cnt_day"], methods=['pearson', 'spearman', 'kendall'], numeric_only=True)

In [ ]:
# Checking normal distribution for sales dataframe item_price, item_cnt_day columns
sales_eda.normal_distr_check(["item_cnt_day", "item_price"], bins=16, edgecolor="coral", color="lightblue")

In [ ]:
# Creating a dataframe with unique store-item pairs 
# for subsequent time series analysis for these pairs
shop_item_gr: pd.DataFrame = pd.DataFrame(sales_df_processed.groupby(["shop_id", "item_id"])["item_cnt_day"].sum())
shop_item_df = shop_item_gr.reset_index() # converting indexes into columns
shop_item_df = shop_item_df[["shop_id", "item_id"]]

In [ ]:
# ts stationarity check for unique shop-item pairs 
import warnings


warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", message="The test statistic is outside of the range of p-values")

nonstationary_shop_item: List[dict] = []
for r in shop_item_df.iterrows():
    shop_id: int = r[1]["shop_id"]
    item_id: int = r[1]["item_id"]

    shop_item_ts: pd.DataFrame = monthly_sales.loc[(monthly_sales["shop_id"] == shop_id) & (monthly_sales["item_id"] == item_id)]
    if shop_item_ts.shape[0] >= 20:
        shop_item_stationarity: tuple = EDA(shop_item_ts).ts_stationarity_check("item_cnt_month")
        if shop_item_stationarity[0] != "stationary": 
            nonstationary_shop_item.append({"shop_id": shop_id, "item_id": item_id, "stationarity" : shop_item_stationarity[0], "tests_stats": {"adf_stats" : shop_item_stationarity[1], "kpss_stats" : shop_item_stationarity[2]}})
            print(f"For {shop_id} shop_id and {item_id} item_id time series is {shop_item_stationarity[0]}.")

In [ ]:
# non-stationarity processing for unique shop-item pairs 
for shop_item in nonstationary_shop_item:
    shop_id: int = shop_item["shop_id"]
    item_id: int = shop_item["item_id"]

    shop_item_ts: pd.DataFrame = monthly_sales.loc[(monthly_sales["shop_id"] == shop_id) & (monthly_sales["item_id"] == item_id)]
    shop_item_ts_etl: ETL = ETL(df=shop_item_ts)
    proc_shop_item_ts: pd.DataFrame = shop_item_ts_etl.ts_nonstatinarity_processing("item_cnt_month")
    monthly_sales.loc[(monthly_sales["shop_id"] == shop_id) & (monthly_sales["item_id"] == item_id), "item_cnt_month"] = proc_shop_item_ts
    print(f"TS for {shop_id} shop and {item_id} item processed.")
    

In [ ]:
# autocorrelation check for unique shop-item pairs 
ac_shop_item: List[dict] = []
for r in shop_item_df.iterrows():
    shop_id: int = r[1]["shop_id"]
    item_id: int = r[1]["item_id"]

    lags_num: int = 3

    shop_item_ts: pd.DataFrame = monthly_sales.loc[(monthly_sales["shop_id"] == shop_id) & (monthly_sales["item_id"] == item_id)]
    if shop_item_ts.shape[0] >= lags_num:
        shop_item_ac_lags = EDA(shop_item_ts).ts_autocorr_check("item_cnt_month", lags_num)
        if len(shop_item_ac_lags) >= 1: 
            ac_shop_item.append({"shop_id": shop_id, "item_id": item_id, "lags": shop_item_ac_lags})
            print(f"For {shop_id} shop_id and {item_id} item_id autocorrelation for {shop_item_ac_lags} lags detected.")

In [ ]:
# ACF-PACF for shop-item sample
shop_item_sample = ac_shop_item[3]

shop_id: int = shop_item_sample["shop_id"]
item_id: int = shop_item_sample["item_id"]
x = monthly_sales.loc[(monthly_sales["shop_id"] == shop_id) & (monthly_sales["item_id"] == item_id), "item_cnt_month"]

sm.graphics.tsa.plot_acf(x.values.squeeze(), lags=20)
plt.show()

In [ ]:
# heteroskedasticity check for monthly_sales DF
EDA(monthly_sales).heterosked_check(["shop_id", "item_id"], "item_cnt_month")

In [ ]:
# monthly data
monthly_sales_sum = monthly_sales.groupby(["month"])["item_cnt_month"].sum()
monthly_sales_sum = pd.DataFrame(monthly_sales_sum).reset_index()
monthly_sales_sum.head()

In [ ]:
plt.plot(monthly_sales_sum["month"].dt.strftime("%Y-%m"), monthly_sales_sum.item_cnt_month)
plt.title("Monthly sales sum", fontsize=14)
plt.xticks(monthly_sales_sum["month"].dt.strftime("%Y-%m")[::2], rotation=45)
plt.show()

In [ ]:
# daily data
sales_df['date'] = pd.to_datetime(sales_df["date"], dayfirst=True)
daily_sales_sum = sales_df.groupby(["date"])["item_cnt_day"].sum()
daily_sales_sum = pd.DataFrame(daily_sales_sum).reset_index()
daily_sales_sum.head()

In [ ]:
plt.plot(daily_sales_sum["date"].dt.strftime("%Y-%m-%d"), daily_sales_sum.item_cnt_day)
plt.title("Daily sales sum", fontsize=14)
plt.xticks(daily_sales_sum["date"].dt.strftime("%Y-%m-%d")[::50], rotation=45)
plt.show()

In [ ]:
# ts monthly sales stationarity check and processing
stationary_monthly_sales_sum: ETL = ETL(df=monthly_sales_sum).ts_nonstatinarity_processing("item_cnt_month")

In [ ]:
monthly_sales_sum["month"] = monthly_sales_sum["month"].astype("int64")
# creating instances of the EDA class for monthly_sales_sum DF
monthly_sales_sum_eda: EDA = EDA(stationary_monthly_sales_sum)

In [ ]:
# ts monthly sales autocorrelation check 
monthly_sales_sum_eda.ts_autocorr_check("item_cnt_month", 20)

In [ ]:
# monthly sales heteroskedasticity check
monthly_sales_sum_eda.heterosked_check(["month"], "item_cnt_month")

In [ ]:
# ts daily sales stationarity check and processing
stationary_daily_sales_sum: ETL = ETL(df=daily_sales_sum).ts_nonstatinarity_processing("item_cnt_day")

In [ ]:
daily_sales_sum["date"] = daily_sales_sum["date"].astype("int64")
# creating instances of the EDA class for daily_sales_sum DF
daily_sales_sum_eda: EDA = EDA(stationary_daily_sales_sum)

In [ ]:
# ts daily sales autocorrelation check 
daily_sales_sum_eda.ts_autocorr_check("item_cnt_day", 20)

In [ ]:
# daily sales heteroskedasticity check
daily_sales_sum_eda.heterosked_check(["date"], "item_cnt_day")